# Schneider Electrick Hackaton - Data Science
## Team 4 - Gerard Jover Pujol - Pol Gràcia Espelt

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Modules import

In [2]:
import os
import sklearn as sk
import pandas as pd
import numpy as np 
import requests as r
import pandas_profiling

### Data load

In [3]:
project_path = "/content/drive/MyDrive/Data_Science_&_AI/NUWE/"
train_data_dir_csv = [project_path + 'Data/CSV/' + x for x in os.listdir(project_path+'Data/CSV') if 'train' in x]
test_data_dir_csv = [project_path + 'Data/CSV/' + x for x in os.listdir(project_path+'Data/CSV') if 'test' in x][0]
train_data_dir_pdf = [project_path + 'Data/pdf/' + x for  x in os.listdir(project_path+'Data/pdf')]
train_data_api = project_path + 'Data/json/links.txt'

#### CSV

In [4]:
train_csv = pd.concat([pd.read_csv(train_data_dir_csv[0]), pd.read_csv(train_data_dir_csv[1], sep = ';')])

In [5]:
train_csv.describe()

,reportingYear,MONTH,DAY,max_wind_speed,avg_wind_speed,min_wind_speed,max_temp,avg_temp,min_temp,DAY WITH FOGS
count,37127.000000,37127.000000,37127.000000,37127.000000,37127.000000,37127.000000,37127.000000,37127.000000,37127.000000,37127.000000
mean,2012.951841,6.487031,14.540335,15.510519,18.013596,22.523559,9.452422,10.441560,13.431015,2.228217
std,3.856172,3.467030,8.128336,3.070188,2.313407,3.064919,5.232214,5.094000,5.225196,3.775751
min,2007.000000,1.000000,1.000000,8.062689,14.000287,15.055647,-3.141464,-0.199176,0.894827,0.000000
25%,2010.000000,3.000000,8.000000,13.315477,16.007158,20.361339,5.855159,7.158428,9.866841,0.000000
50%,2013.000000,7.000000,15.000000,15.503568,18.006838,22.553985,9.734382,10.733914,13.723792,1.000000
75%,2016.000000,9.000000,22.000000,17.708038,20.013530,24.713992,13.287165,14.162492,17.235091,2.000000
max,2020.000000,12.000000,28.000000,22.991382,21.999973,29.914367,20.855885,19.999403,24.855425,19.000000


### api

In [6]:
with open(train_data_api) as file:
    links = [link.strip() for link in file]

int_cols = ['MONTH','reportingYear', 'DAY', 'max_wind_speed', 'avg_wind_speed', 'min_wind_speed', 'max_temp', 'min_temp', 'avg_temp', 'DAY WITH FOGS']
json_df = pd.DataFrame()
for link in links:
    data_json = r.get(link).json()
    tmp_df = pd.json_normalize(data_json)
    #set index
    tmp_df.set_index(tmp_df[''], inplace = True)
    tmp_df.drop('', axis = 1, inplace=True)
    #values to int
    tmp_df[int_cols] = tmp_df[int_cols].apply(pd.to_numeric)
    #print(tmp_df.columns)
    #concat
    json_df = pd.concat([json_df, tmp_df])
    #train_csv = pd.concat([train_csv, tmp_df])

In [7]:
c = 0
l = list(json_df['facilityName'])
for x in train_csv.facilityName:
    if x not in l:
        c+=1

### Combine csv and json

In [8]:
df_bo = pd.concat([train_csv, json_df])

In [9]:
df_bo.isna().sum()

countryName                         0
eprtrSectorName                     0
EPRTRAnnexIMainActivityLabel        0
FacilityInspireID                   0
facilityName                        0
City                                0
targetRelease                       0
pollutant                           0
reportingYear                       0
MONTH                               0
DAY                                 0
CONTINENT                           0
max_wind_speed                      0
avg_wind_speed                      0
min_wind_speed                      0
max_temp                            0
avg_temp                            0
min_temp                            0
DAY WITH FOGS                       0
REPORTER NAME                       0
CITY ID                             0
EPRTRAnnexIMainActivityCode     37127
EPRTRSectorCode                 37127
dtype: int64

### Replace nans

In [10]:
label2code = pd.DataFrame(df_bo[['EPRTRAnnexIMainActivityLabel', 'EPRTRAnnexIMainActivityCode', 'EPRTRSectorCode']].value_counts()).reset_index()
label2code

,EPRTRAnnexIMainActivityLabel,EPRTRAnnexIMainActivityCode,EPRTRSectorCode,0
0,Thermal power stations and other combustion in...,1(c),1,9378
1,Landfills (excluding landfills of inert waste ...,5(d),5,4481
2,Installations for the incineration of non-haza...,5(b),5,1552
3,Installations for the production of cement cli...,3(c)(i),3,1504
4,"Installations for the manufacture of glass, in...",3(e),3,1213
...,...,...,...,...
65,Industrial plants for the preservation of wood...,6(c),6,5
66,Chemical installations for the production on a...,4(b)(iii),4,5
67,Installations for the production on an industr...,4(f),4,3
68,"Installations for the building of, and paintin...",9(e),9,3


In [11]:
label2SC = dict(zip(list(label2code.EPRTRAnnexIMainActivityLabel), list(label2code.EPRTRSectorCode)))
label2MAC = dict(zip(list(label2code.EPRTRAnnexIMainActivityLabel), list(label2code.EPRTRAnnexIMainActivityCode)))

In [12]:
f = lambda x: label2SC[x] if x != 'Chemical installations for the production on an industrial scale of basic organic chemicals: Organometallic compounds' else np.nan
df_bo['EPRTRSectorCode'] = df_bo['EPRTRAnnexIMainActivityLabel'].apply(f)

In [13]:
f = lambda x: label2MAC[x] if x != 'Chemical installations for the production on an industrial scale of basic organic chemicals: Organometallic compounds' else np.nan
df_bo['EPRTRAnnexIMainActivityCode'] = df_bo['EPRTRAnnexIMainActivityLabel'].apply(f)

In [14]:
df_bo.isna().sum()

countryName                     0
eprtrSectorName                 0
EPRTRAnnexIMainActivityLabel    0
FacilityInspireID               0
facilityName                    0
City                            0
targetRelease                   0
pollutant                       0
reportingYear                   0
MONTH                           0
DAY                             0
CONTINENT                       0
max_wind_speed                  0
avg_wind_speed                  0
min_wind_speed                  0
max_temp                        0
avg_temp                        0
min_temp                        0
DAY WITH FOGS                   0
REPORTER NAME                   0
CITY ID                         0
EPRTRAnnexIMainActivityCode     2
EPRTRSectorCode                 2
dtype: int64

### pdf

In [15]:
!pip install pdfplumber

In [16]:
import pdfplumber

def pdf_2_dict(pdf_path):
    # Extract text from pdf file
    with pdfplumber.open(pdf_path) as temp:
        first_page = temp.pages[0]
        text = first_page.extract_text()
        text_list = text.split("\n")

    # Create empty dict
    pdf_dict = {}

    # Generate new fields
    pdf_dict["id"] = int(text_list[1].split()[1])
    pdf_dict["facilityName"] = " ".join(text_list[2].split()[2:-1])
    pdf_dict["FacilityInspireID"] = text_list[3].split()[1]
    pdf_dict["countryName"] = " ".join(text_list[4].split("CONTINENT:")[0].split()[1:])
    pdf_dict["CONTINENT"] = " ".join(text_list[4].split("CONTINENT:")[1:])[1:]
    pdf_dict["City"] = " ".join(text_list[5].split()[1:])
    pdf_dict["EPRTRSectorCode"] = int(" ".join(text_list[6].split("eprtrSectorName:")[0].split()[1:]))
    pdf_dict["eprtrSectorName"] = " ".join(text_list[6].split("eprtrSectorName:")[1:])[1:]
    pdf_dict["EPRTRAnnexIMainActivityCode"] = text_list[7].split(":")[1]
    pdf_dict["targetRelease"] = " ".join(text_list[8].split("pollutant:")[0].split()[1:])
    pdf_dict["pollutant"] = " ".join(text_list[8].split("pollutant:")[1:])[1:]
    pdf_dict["emissions"] = float(text_list[9].split()[1].replace(",", "."))
    pdf_dict["DAY"] = int(text_list[10].split()[1])
    pdf_dict["MONTH"] = int(text_list[10].split()[3])
    pdf_dict["reportingYear"] = int(text_list[10].split()[5])
    pdf_dict["max_wind_speed"] = float(text_list[12].split()[1].replace(",", "."))
    pdf_dict["min_wind_speed"] = float(text_list[12].split()[3].replace(",", "."))
    pdf_dict["avg_wind_speed"] = float(text_list[12].split()[5].replace(",", "."))
    pdf_dict["max_temp"] = float(text_list[13].split()[1].replace(",", "."))
    pdf_dict["min_temp"] = float(text_list[13].split()[3].replace(",", "."))
    pdf_dict["avg_temp"] = float(text_list[13].split()[5].replace(",", "."))
    pdf_dict["DAY WITH FOGS"] = int(text_list[14].split()[-1])
    pdf_dict["REPORTER NAME"] = text_list[15].split("REPORTER NAME:")[-1]
    pdf_dict["CITY ID"] = text_list[16].split("CITY_ID ")[-1]

    return pdf_dict

In [17]:
pdf_path_dir = "/content/drive/MyDrive/Data_Science_&_AI/NUWE/Data/pdf/"
pdf_path_list = os.listdir(pdf_path_dir)

df_pdf = pd.DataFrame()
for pdf_path in pdf_path_list:
    pdf_dict = pdf_2_dict(pdf_path_dir + pdf_path)
    pd_series = pd.Series(pdf_dict)
    df_pdf = df_pdf.append(pd_series, ignore_index=True)

### combine csv, json and pdf

In [25]:
df_final = pd.concat([df_bo, df_pdf])

### remove columns

In [26]:
drop_cols = ["id", "emissions", "EPRTRAnnexIMainActivityLabel", "targetRelease", "CONTINENT", "CITY ID"]
df_final = df_final.drop(drop_cols, axis=1)

### dtypes

In [27]:
from pandas.api.types import is_string_dtype
from pandas.api.types import is_numeric_dtype

In [28]:
for col in df_final.columns:
    if col != "pollutant":
        if is_string_dtype(df_final[col]):
            df_final[col] = df_final[col].astype('category').cat.codes
        if is_numeric_dtype(df_final[col]):
            df_final[col] = df_final[col].astype(np.int32)

### target

In [29]:
from sklearn.preprocessing import OrdinalEncoder


ord_enc = OrdinalEncoder()
df_final["pollutant"] = ord_enc.fit_transform(df_final[["pollutant"]])
#ord_enc.inverse_transform

In [34]:
df_final

,countryName,eprtrSectorName,FacilityInspireID,facilityName,City,pollutant,reportingYear,MONTH,DAY,max_wind_speed,avg_wind_speed,min_wind_speed,max_temp,avg_temp,min_temp,DAY WITH FOGS,REPORTER NAME,EPRTRAnnexIMainActivityCode,EPRTRSectorCode
0,10,4,6787,3390,4139,0.0,2015,10,20,15,14,21,2,4,9,2,32063,20,7
1,15,4,3413,6713,4366,2.0,2018,9,21,19,19,21,5,7,12,1,3650,19,7
2,28,8,1328,1439,3349,1.0,2019,2,4,12,14,17,1,4,8,2,44039,50,9
3,5,2,565,2634,2119,2.0,2012,8,6,11,16,17,10,10,15,0,6831,2,5
4,8,8,6222,6879,4453,1.0,2018,12,22,17,20,21,11,11,16,2,4684,52,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77,31,2,6215,3555,1718,2.0,2019,1,19,-2147483648,-2147483648,-2147483648,-2147483648,-2147483648,-2147483648,13,23968,2,0
78,31,8,6159,6824,4304,1.0,2019,4,9,-2147483648,-2147483648,-2147483648,-2147483648,-2147483648,-2147483648,5,37889,50,3
79,31,8,6161,740,3405,1.0,2019,7,18,-2147483648,-2147483648,-2147483648,-2147483648,-2147483648,-2147483648,4,30533,49,3
80,31,8,6199,1990,1264,1.0,2019,1,22,-2147483648,-2147483648,-2147483648,-2147483648,-2147483648,-2147483648,2,22153,50,3


In [35]:
df_final.to_csv("df_final.csv")

### Models

### Training

### Testing